<a href="https://colab.research.google.com/github/CaiqueBorges/Projeto_cats-dogs/blob/main/Cats_dogs_ensembles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. OBTER DATASET

In [1]:
!apt-get install subversion
!svn export https://github.com/CaiqueBorges/Projeto_cats-dogs/trunk/images_cats-dogs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libapr1 libaprutil1 libserf-1-1 libsvn1 libutf8proc2
Suggested packages:
  db5.3-util libapache2-mod-svn subversion-tools
The following NEW packages will be installed:
  libapr1 libaprutil1 libserf-1-1 libsvn1 libutf8proc2 subversion
0 upgraded, 6 newly installed, 0 to remove and 15 not upgraded.
Need to get 2,672 kB of archives.
After this operation, 10.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libapr1 amd64 1.7.0-8ubuntu0.22.04.1 [108 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libaprutil1 amd64 1.6.1-5ubuntu4.22.04.2 [92.8 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libserf-1-1 amd64 1.3.9-10ubuntu2 [50.0 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libutf8proc2 amd64 2.7.0-3 [73.9 kB]
Get:5 http://archive.

In [2]:
## Importando as libs necessárias
from google.colab import files
import os, io, time

pasta_alvo = 'images_cats-dogs'

# Lista para armazenar os nomes dos arquivos
uploaded_images = []

# Verifica se a pasta existe
if os.path.exists(pasta_alvo) and os.path.isdir(pasta_alvo):
  # Percorre a pasta e obtém os nomes dos arquivos
  for nome_arquivo in os.listdir(pasta_alvo):
    if nome_arquivo.lower().endswith('.jpg'):
      uploaded_images.append(nome_arquivo)
else:
  print(f"A pasta {pasta_alvo} não existe.")

# 2 HOG COM PCA


## 2.1 Aplicando HOG e PCA

In [9]:
## Reduzir as imagens
from skimage.io import imread, imshow
from skimage.transform import resize
from skimage.feature import hog
## pandas
import pandas as pd
## Importando PCA do SkLearn
from sklearn.decomposition import PCA

def classify_animal(value):
  if value in ['Abyssinian', 'Persian']:
    return 'cat'
  else:
    return 'dog'

hogs = [16,20,24]
pcas = [.50,.75,.95,1]


hogs_pcas = []

def hog_f(hog_size):
  hog_features = []
  #for filename in uploaded_images.keys():
  for filename in uploaded_images:
    image = imread('images_cats-dogs/'+ filename)
    image_resized = resize(image, (128,128))
    fd, hog_image = hog(image_resized, orientations=9, pixels_per_cell=(hog_size, hog_size),
                      cells_per_block=(2, 2), visualize=True, channel_axis=-1)
    hog_features.append(fd)

  ## Adicionar as colunas com os valores do hog para cada um
  #data = {"pathfile": uploaded_images.keys(), "hog_features": hog_features}
  data = {"pathfile": uploaded_images, "hog_features": hog_features}
  df = pd.DataFrame(data)
  df['classe'] = df['pathfile'].str.split('_').str[0].apply(classify_animal)
  df = df.drop('pathfile', axis=1)

  # transformando cada valor do hog_feature em um atributo (coluna)
  df2 = pd.DataFrame(df['hog_features'].tolist())
  df2.columns = df2.columns.map(lambda x: f'hog_feature_{x+1}')
  df = pd.concat([df.drop('hog_features', axis=1), df2], axis=1)
  return df

for hog_size in hogs:
  df = hog_f(hog_size)
  df.to_csv(f'HOG {hog_size}x{hog_size}.csv', index=False)
  hogs_pcas.append([f'hog {hog_size}', df])
  print(f'hog de {hog_size}x{hog_size}')
  print(f'shape: {df.shape}')
  print()

  # ----- COMEÇANDO PCA ------
  for porcent in pcas:
    y = df.classe # Target variable (classe)
    X = df.copy().drop('classe', axis=1) # Features

    ##pca_obj = PCA(n_components=4) #para dizer quantos componentes quero ter
    pca_obj = PCA(n_components=porcent, whiten=True) #qual porcentagem de informação eu quero
    pca_result = pca_obj.fit_transform(X)

    print(f'hog de {hog_size}x{hog_size} e pca de {porcent}')
    print(f'shape: {pca_result.shape}')
    print()

    colunas = []
    for i in range(pca_result.shape[1]):
      colunas.append('pca_'+str(i))

    pca_dataset = pd.DataFrame(data = pca_result, columns = colunas) # Atual

    ## Juntando o atributo classe
    final_data = pca_dataset.join(y)

    # Salvando Pessoa.csv transformado
    df_hog_pca = pd.DataFrame(final_data)
    df_hog_pca.to_csv(f'HOG {hog_size}x{hog_size}_pca-{porcent}.csv', index=False)

    # Importando arquivo transformado
    #from google.colab import files
    #files.download(f'HOG {hog_size}x{hog_size}_pca-{porcent}.csv')
    hogs_pcas.append([f'hog {hog_size} - pca {porcent}', df_hog_pca])


hog de 16x16
shape: (800, 1765)

hog de 16x16 e pca de 0.5
shape: (800, 39)

hog de 16x16 e pca de 0.75
shape: (800, 111)

hog de 16x16 e pca de 0.95
shape: (800, 313)

hog de 16x16 e pca de 1
shape: (800, 1)

hog de 20x20
shape: (800, 901)

hog de 20x20 e pca de 0.5
shape: (800, 25)

hog de 20x20 e pca de 0.75
shape: (800, 69)

hog de 20x20 e pca de 0.95
shape: (800, 200)

hog de 20x20 e pca de 1
shape: (800, 1)

hog de 24x24
shape: (800, 577)

hog de 24x24 e pca de 0.5
shape: (800, 19)

hog de 24x24 e pca de 0.75
shape: (800, 49)

hog de 24x24 e pca de 0.95
shape: (800, 143)

hog de 24x24 e pca de 1
shape: (800, 1)



# 3. CNN

## 3.1 Aplicando CNN com max e avg

In [4]:
from google.colab import files
## Carregar as Libs necessarias
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16, preprocess_input
#pandas e numpy
import numpy as np
import pandas as pd
## Carregar e processar imagem
from skimage import io
from skimage.transform import resize
## Importando PCA do SkLearn
from sklearn.decomposition import PCA
import cv2
#Para evitar erros caso não seja RGB
from PIL import Image
import tempfile


def cnn_pca(model, list_files, cnn_size):
  cnn_features = []

  for filename in list_files:
    print(filename)
    img = imread('images_cats-dogs/'+filename)
    image_resized = resize(img, (128,128))
    ## Colocando a imagem no formato correto
    x = image.img_to_array(image_resized)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    ## Extração de características

    try:
      # Tentar extrair características
      features = model.predict(x)
    except Exception as e:
      # Tratar a exceção (converter a imagem para RGB)
      image_pil = Image.open('images_cats-dogs/'+filename)
      image_pil = image_pil.convert("RGB")
      # Salvar a imagem temporariamente
      temp_filename = tempfile.NamedTemporaryFile(suffix=".jpg", delete=False).name
      image_pil.save(temp_filename)
      imagem_rgb = io.imread(temp_filename)
      #retornando agora com a imagem rgb
      image_resized_rgb = resize(imagem_rgb, (128, 128))
      x = image.img_to_array(image_resized_rgb)
      x = np.expand_dims(x, axis=0)
      x = preprocess_input(x)
      features = model.predict(x)

    ## Tornar as caracteristicas em forma de vetor
    features_flatten = features.flatten()

    cnn_features.append(features_flatten)

  ## Adicionar as colunas com os valores do cnn para cada um
  data = {"pathfile": list_files, "cnn_features": cnn_features}
  df = pd.DataFrame(data)
  df['classe'] = df['pathfile'].str.split('_').str[0].apply(classify_animal)
  df = df.drop('pathfile', axis=1)

  # transformando cada valor do cnn_feature em um atributo (coluna)
  df2 = pd.DataFrame(df['cnn_features'].tolist())
  df2.columns = df2.columns.map(lambda x: f'cnn_feature_{x+1}')
  df = pd.concat([df.drop('cnn_features', axis=1), df2], axis=1)

  # Salvando transformado
  df.to_csv(f'CNN_{cnn_size}.csv', index=False)

  # Importando arquivo transformado
  files.download(f'CNN_{cnn_size}.csv')
  return [cnn_size, df]

lista_cnn = []
## Carregar o modelo de CNN pre-treinado e aplicar
model = VGG16(weights='imagenet', include_top=False, pooling='max')
lista_cnn.append(cnn_pca(model, uploaded_images, 'VGG16_max'))

model = VGG16(weights='imagenet', include_top=False, pooling='avg')
lista_cnn.append(cnn_pca(model, uploaded_images, 'VGG16_avg'))

58889256/58889256 [==============================] - 0s 0us/step
leonberger_120.jpg
1/1 [==============================] - 0s 433ms/step
Abyssinian_111.jpg
1/1 [==============================] - 0s 178ms/step
leonberger_161.jpg
1/1 [==============================] - 0s 181ms/step
Persian_36.jpg
1/1 [==============================] - 0s 178ms/step
Persian_160.jpg
1/1 [==============================] - 0s 177ms/step
Abyssinian_206.jpg
1/1 [==============================] - 0s 177ms/step
leonberger_198.jpg
1/1 [==============================] - 0s 172ms/step
Persian_129.jpg
1/1 [==============================] - 0s 170ms/step
leonberger_114.jpg
1/1 [==============================] - 0s 177ms/step
Persian_217.jpg
1/1 [==============================] - 0s 170ms/step
Abyssinian_51.jpg
1/1 [==============================] - 0s 172ms/step
leonberger_81.jpg
1/1 [==============================] - 0s 182ms/step
Abyssinian_89.jpg
1/1 [==============================] - 0s 173ms/step
wheaten_terrier

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

leonberger_120.jpg
1/1 [==============================] - 0s 293ms/step
Abyssinian_111.jpg
1/1 [==============================] - 0s 176ms/step
leonberger_161.jpg
1/1 [==============================] - 0s 172ms/step
Persian_36.jpg
1/1 [==============================] - 0s 179ms/step
Persian_160.jpg
1/1 [==============================] - 0s 171ms/step
Abyssinian_206.jpg
1/1 [==============================] - 0s 175ms/step
leonberger_198.jpg
1/1 [==============================] - 0s 174ms/step
Persian_129.jpg
1/1 [==============================] - 0s 172ms/step
leonberger_114.jpg
1/1 [==============================] - 0s 174ms/step
Persian_217.jpg
1/1 [==============================] - 0s 170ms/step
Abyssinian_51.jpg
1/1 [==============================] - 0s 172ms/step
leonberger_81.jpg
1/1 [==============================] - 0s 181ms/step
Abyssinian_89.jpg
1/1 [==============================] - 0s 179ms/step
wheaten_terrier_20.jpg
1/1 [==============================] - 0s 177ms/step
Abys

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [5]:
len(hogs_pcas)

15

In [8]:
len(lista_cnn)

2

In [10]:
bases = hogs_pcas + lista_cnn
len(bases)

17

# 4. Comitês

In [13]:
## Carregando o Bagging, Boosting e RandomForest
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import StackingClassifier
##Carregando os algoritmos base (DT), (MLP), (k-NN) e (NB)
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
#métricas
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_predict, train_test_split, KFold, cross_val_score

## 4.1 Bagging

In [6]:
def train_bagging(name_estimator, estimator, n_estimators,
                  X, y, X_train_70, y_train_70, X_test_30):
  bgclassifier = BaggingClassifier(estimator=estimator, n_estimators=n_estimators)
  bgclassifier.fit(X_train_70,y_train_70)
  y_pred = bgclassifier.predict(X_test_30)
  ## Model Accuracy Hodut
  acuracia = metrics.accuracy_score(y_test_30, y_pred)
  # 10-fold CV
  kf = KFold(n_splits=10, random_state=1, shuffle=True)
  scores = cross_val_score(bgclassifier, X, y, scoring='accuracy', cv=kf)
  return acuracia, scores


In [6]:
estimator = [DecisionTreeClassifier()]

for b in bases:
  base_name = b[0]
  base = b[1]
  y = base.classe
  X = base.copy().drop('classe', axis=1)
  X_train_70, X_test_30, y_train_70, y_test_30 = train_test_split(X, y, test_size=0.3, random_state=73)


## 4.2 Boosting

## 4.3 Random Forest

## 4.4 Stacking